In [11]:
import numpy as np
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from scipy.stats import randint
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [12]:
with open('Datasets/train.json', 'r') as file:
        train_data = json.load(file)

pd.DataFrame(train_data)

,audio_embedding,is_turkey,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[172, 34, 216, 110, 208, 46, 95, 66, 161, 125...",0,kDCk3hLIVXo,70,60
1,"[[169, 20, 165, 102, 205, 62, 110, 103, 211, 1...",1,DPcGzqHoo7Y,40,30
2,"[[148, 8, 138, 60, 237, 48, 121, 108, 145, 177...",1,7yM63MTHh5k,240,230
3,"[[151, 0, 162, 88, 171, 71, 47, 90, 179, 190, ...",1,luG3RmUAxxM,520,510
4,"[[162, 17, 187, 111, 211, 105, 92, 67, 203, 15...",0,PIm3cjxTpOk,10,0
...,...,...,...,...,...
1190,"[[90, 105, 224, 173, 117, 88, 119, 2, 219, 59,...",0,AKIWjFmAMt8,30,20
1191,"[[132, 154, 242, 164, 181, 58, 154, 63, 163, 4...",0,QTRCw0xn6uc,40,30
1192,"[[148, 8, 171, 123, 241, 91, 126, 154, 96, 246...",1,DbJJoQfo3kc,40,30
1193,"[[166, 13, 201, 135, 190, 93, 71, 30, 139, 151...",0,opqJ1SZmyGk,200,190


In [13]:
with open('Datasets/test.json', 'r') as file:
        test_data = json.load(file)

pd.DataFrame(test_data)

,audio_embedding,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[177, 20, 226, 132, 198, 81, 111, 59, 132, 18...",pyKh38FXD3E,10,0
1,"[[169, 21, 204, 161, 195, 72, 60, 39, 152, 184...",THhP1idrWXA,40,30
2,"[[165, 13, 198, 141, 199, 81, 173, 54, 119, 11...",jsw3T6GY2Nw,40,30
3,"[[167, 18, 188, 159, 198, 63, 156, 36, 179, 22...",nFkXTMHcjMU,24,14
4,"[[178, 32, 181, 100, 198, 46, 82, 83, 136, 227...",Au8g9kAlrLQ,40,30
...,...,...,...,...
1191,"[[153, 26, 118, 69, 255, 87, 153, 59, 125, 204...",lQOQR6HCGyE,11,1
1192,"[[159, 8, 163, 128, 172, 64, 36, 56, 110, 236,...",CqhPMjumTOA,40,30
1193,"[[78, 56, 132, 20, 92, 120, 6, 133, 191, 156, ...",Ccn2LQJc2MI,40,30
1194,"[[81, 115, 208, 104, 236, 91, 95, 71, 161, 152...",f6OhmrIdB-g,40,30


In [14]:
train_data = pd.DataFrame(train_data)
print(train_data)

                                        audio_embedding  is_turkey  \
0     [[172, 34, 216, 110, 208, 46, 95, 66, 161, 125...          0   
1     [[169, 20, 165, 102, 205, 62, 110, 103, 211, 1...          1   
2     [[148, 8, 138, 60, 237, 48, 121, 108, 145, 177...          1   
3     [[151, 0, 162, 88, 171, 71, 47, 90, 179, 190, ...          1   
4     [[162, 17, 187, 111, 211, 105, 92, 67, 203, 15...          0   
...                                                 ...        ...   
1190  [[90, 105, 224, 173, 117, 88, 119, 2, 219, 59,...          0   
1191  [[132, 154, 242, 164, 181, 58, 154, 63, 163, 4...          0   
1192  [[148, 8, 171, 123, 241, 91, 126, 154, 96, 246...          1   
1193  [[166, 13, 201, 135, 190, 93, 71, 30, 139, 151...          0   
1194  [[165, 45, 191, 96, 255, 87, 97, 77, 152, 215,...          0   

           vid_id  end_time_seconds_youtube_clip  \
0     kDCk3hLIVXo                             70   
1     DPcGzqHoo7Y                             40   
2  

In [15]:
test_data = pd.DataFrame(test_data)
print(test_data)

                                        audio_embedding       vid_id  \
0     [[177, 20, 226, 132, 198, 81, 111, 59, 132, 18...  pyKh38FXD3E   
1     [[169, 21, 204, 161, 195, 72, 60, 39, 152, 184...  THhP1idrWXA   
2     [[165, 13, 198, 141, 199, 81, 173, 54, 119, 11...  jsw3T6GY2Nw   
3     [[167, 18, 188, 159, 198, 63, 156, 36, 179, 22...  nFkXTMHcjMU   
4     [[178, 32, 181, 100, 198, 46, 82, 83, 136, 227...  Au8g9kAlrLQ   
...                                                 ...          ...   
1191  [[153, 26, 118, 69, 255, 87, 153, 59, 125, 204...  lQOQR6HCGyE   
1192  [[159, 8, 163, 128, 172, 64, 36, 56, 110, 236,...  CqhPMjumTOA   
1193  [[78, 56, 132, 20, 92, 120, 6, 133, 191, 156, ...  Ccn2LQJc2MI   
1194  [[81, 115, 208, 104, 236, 91, 95, 71, 161, 152...  f6OhmrIdB-g   
1195  [[164, 21, 189, 141, 198, 32, 151, 69, 129, 16...  vCxGJelmAeg   

      end_time_seconds_youtube_clip  start_time_seconds_youtube_clip  
0                                10                             

In [ ]:
# KẾT QUẢ CAO NHẤT
# Tối ưu Random Forest & GridSearchCV
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score, precision_score,
    recall_score, f1_score, accuracy_score
)

# Tạo ma trận đặc trưng và nhãn
train_X = np.stack(train_data['audio_embedding'].apply(lambda x: np.mean(x, axis=0)))
train_Y = train_data['is_turkey'].values

# Chỉ chọn test_data có embedding hợp lệ
valid_idx = test_data['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
test_X = np.stack(test_data.loc[valid_idx, 'audio_embedding'].apply(lambda x: np.mean(x, axis=0)))

# Chuẩn hóa
scaler = StandardScaler()
Z = scaler.fit_transform(train_X)
test_Z = scaler.transform(test_X)

# Tách tập train/val
train_Z, val_Z, train_Y, val_Y = train_test_split(Z, train_Y, test_size=0.3, random_state=42)

# Cài đặt mô hình và grid search
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 3],
    'class_weight': ['balanced']
}

base_model = RandomForestClassifier(random_state=42, n_jobs=-1)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(base_model, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=1)

# Tìm mô hình tốt nhất
grid_search.fit(train_Z, train_Y)

# Lấy model tốt nhất
best_model = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)

# Dự đoán trên tập validation
y_pred_prob = best_model.predict_proba(val_Z)[:, 1]
y_pred = best_model.predict(val_Z)

# Đánh giá mô hình
print("AUC Score   :", roc_auc_score(val_Y, y_pred_prob))
print("Accuracy    :", accuracy_score(val_Y, y_pred))
print("Precision   :", precision_score(val_Y, y_pred))
print("Recall      :", recall_score(val_Y, y_pred))
print("F1 Score    :", f1_score(val_Y, y_pred))

# Dự đoán trên test
test_pred_prob = best_model.predict_proba(test_Z)[:, 1]

# Gán kết quả
test_data['is_turkey'] = np.nan
test_data.loc[valid_idx, 'is_turkey'] = test_pred_prob

# Lưu kết quả
test_data[['vid_id', 'is_turkey']].to_csv('result.csv', index=False)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best parameters: {'class_weight': 'balanced', 'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
AUC Score   : 0.9836122078084852
Accuracy    : 0.9164345403899722
Precision   : 0.952054794520548
Recall      : 0.8580246913580247
F1 Score    : 0.9025974025974026


In [17]:
# Tối ưu Random Forest
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split   
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score, precision_score,
    recall_score, f1_score, accuracy_score
)

# Tạo ma trận đặc trưng và nhãn
train_X = np.stack(train_data['audio_embedding'].apply(lambda x: np.mean(x, axis=0)))
train_Y = train_data['is_turkey'].values

# Chỉ chọn test_data có embedding hợp lệ
valid_idx = test_data['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
test_X = np.stack(test_data.loc[valid_idx, 'audio_embedding'].apply(lambda x: np.mean(x, axis=0)))

# Chuẩn hóa
scaler = StandardScaler()
Z = scaler.fit_transform(train_X)
test_Z = scaler.transform(test_X)

# Tách tập train/val
train_Z, val_Z, train_Y, val_Y = train_test_split(Z, train_Y, test_size=0.3, random_state=42)

# Cấu hình RandomForest tối ưu hơn
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=3,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

# Huấn luyện
model.fit(train_Z, train_Y)

# Dự đoán
y_pred_prob = model.predict_proba(val_Z)[:, 1]
y_pred = model.predict(val_Z)

# Đánh giá mô hình
print("AUC Score   :", roc_auc_score(val_Y, y_pred_prob))
print("Accuracy    :", accuracy_score(val_Y, y_pred))
print("Precision   :", precision_score(val_Y, y_pred))
print("Recall      :", recall_score(val_Y, y_pred))
print("F1 Score    :", f1_score(val_Y, y_pred))

# Dự đoán trên test
test_pred_prob = model.predict_proba(test_Z)[:, 1]

# Gán kết quả
test_data['is_turkey'] = np.nan
test_data.loc[valid_idx, 'is_turkey'] = test_pred_prob

# Lưu kết quả
test_data[['vid_id', 'is_turkey']].to_csv('result.csv', index=False)


AUC Score   : 0.9817321551670113
Accuracy    : 0.9080779944289693
Precision   : 0.9271523178807947
Recall      : 0.8641975308641975
F1 Score    : 0.8945686900958466


In [18]:
# BASIC - Random Forest
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split   
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

train_X = np.stack(train_data['audio_embedding'].apply(lambda x: np.mean(x, axis=0)))
train_Y = train_data['is_turkey'].values

# Chỉ chọn những dòng có audio_embedding hợp lệ
valid_idx = test_data['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)


test_X = np.stack(test_data.loc[valid_idx, 'audio_embedding'].apply(lambda x: np.mean(x, axis=0)))


scaler = StandardScaler()
Z = scaler.fit_transform(train_X)
test_Z = scaler.transform(test_X)


train_Z, val_Z, train_Y, val_Y = train_test_split(Z, train_Y, test_size=0.3, random_state=42)


model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(train_Z, train_Y)


y_pred_prob = model.predict_proba(val_Z)[:, 1]
print("AUC score:", roc_auc_score(val_Y, y_pred_prob))


test_pred_prob = model.predict_proba(test_Z)[:, 1]


test_data['is_turkey'] = np.nan
test_data.loc[valid_idx, 'is_turkey'] = test_pred_prob

# (Tuỳ chọn) Nếu muốn thay NaN bằng 0.5 hoặc 0
# test_data['is_turkey'].fillna(0.5, inplace=True)
test_data[['vid_id', 'is_turkey']].to_csv('result.csv', index=False)

AUC score: 0.9815911512189007


In [19]:
print(test_data.columns.tolist())

['audio_embedding', 'vid_id', 'end_time_seconds_youtube_clip', 'start_time_seconds_youtube_clip', 'is_turkey']


In [ ]:
# SO SÁNH NHIỀU MODELS: 
# KẾT QUẢ: RandomForest tốt nhất 






# import numpy as np
# import pandas as pd
# import warnings

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import GaussianNB
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import (
#     roc_auc_score, precision_score, recall_score,
#     f1_score, accuracy_score
# )

# from lightgbm import LGBMClassifier
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier

# # Tắt toàn bộ cảnh báo
# warnings.filterwarnings("ignore")

# # Chuẩn bị dữ liệu huấn luyện
# train_X = np.stack(train_data['audio_embedding'].apply(lambda x: np.mean(x, axis=0)))
# train_Y = train_data['is_turkey'].values

# # Lọc những dòng test có audio_embedding hợp lệ
# valid_idx = test_data['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
# test_X = np.stack(test_data.loc[valid_idx, 'audio_embedding'].apply(lambda x: np.mean(x, axis=0)))

# # Chuẩn hóa dữ liệu
# scaler = StandardScaler()
# Z = scaler.fit_transform(train_X)
# test_Z = scaler.transform(test_X)

# # Chia train / validation
# train_Z, val_Z, train_Y, val_Y = train_test_split(Z, train_Y, test_size=0.3, random_state=42)

# # Các mô hình cần đánh giá
# models = {
#     'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
#     'LogisticRegression': LogisticRegression(max_iter=1000, random_state=42),
#     'GaussianNB': GaussianNB(),
#     'LightGBM': LGBMClassifier(random_state=42, verbose=-1),
#     'XGBoost': XGBClassifier(verbosity=0, random_state=42),
#     'CatBoost': CatBoostClassifier(verbose=0, random_seed=42)
# }

# # Đánh giá từng mô hình
# for name, model in models.items():
#     model.fit(train_Z, train_Y)

#     # Với LightGBM cần convert về DataFrame để tránh cảnh báo
#     if name == 'LightGBM':
#         val_input = pd.DataFrame(val_Z)
#     else:
#         val_input = val_Z

#     val_pred_prob = model.predict_proba(val_input)[:, 1]
#     val_pred = model.predict(val_input)

#     auc = roc_auc_score(val_Y, val_pred_prob)
#     precision = precision_score(val_Y, val_pred)
#     recall = recall_score(val_Y, val_pred)
#     f1 = f1_score(val_Y, val_pred)
#     accuracy = accuracy_score(val_Y, val_pred)

#     print(f"\n{name} Results:")
#     print(f"AUC Score     : {auc:.4f}")
#     print(f"Precision     : {precision:.4f}")
#     print(f"Recall        : {recall:.4f}")
#     print(f"F1 Score      : {f1:.4f}")
#     print(f"Accuracy      : {accuracy:.4f}")

# # Chọn mô hình tốt nhất, ví dụ: LightGBM
# final_model = models['LightGBM']
# final_test_input = pd.DataFrame(test_Z)
# test_pred_prob = final_model.predict_proba(final_test_input)[:, 1]

# # Gán kết quả vào test_data
# test_data['is_turkey'] = np.nan
# test_data.loc[valid_idx, 'is_turkey'] = test_pred_prob

# # Ghi kết quả ra file
# test_data[['vid_id', 'is_turkey']].to_csv('result.csv', index=False)
